# GANART: an AI that generates MtG Art

In [ ]:
pip install torch===1.4.0 torchvision===0.4.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
pip install torchvision tensorboardx matplotlib numpy

In [ ]:
from IPython import display

import random
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

from utils import Logger

### DATA and MISC

In [ ]:
device = torch.device('cuda:0')

In [ ]:
def noise(size):
    n = Variable(torch.randn(size, 100))
    if torch.cuda.is_available(): return n.cuda()
    return n.to(device)

### GENERATOR

In [ ]:
class GeneratorNet(torch.nn.Module):
    def __init__(self):
        super(GeneratorNet, self).__init__()
        
        self.linear = torch.nn.Linear(100, 1024*4*4)
        
        self.conv1 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=1024, out_channels=512, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=512, out_channels=256, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=256, out_channels=128, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=128, out_channels=3, kernel_size=4,
                stride=2, padding=1, bias=False
            )
        )
        self.out = torch.nn.Tanh()

    def forward(self, x):
        # Project and reshape
        x = self.linear(x)
        x = x.view(x.shape[0], 1024, 4, 4)
        # Convolutional layers
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        # Apply Tanh
        return self.out(x)

In [ ]:
G = GeneratorNet()
G.load_state_dict(torch.load('./results/models/basics/G_epoch_300'))
G.to(device)
G.eval()

#### Tests

In [ ]:
num_test_samples = 16
test_noise = noise(num_test_samples)

In [ ]:
num_tests = 5
for i in range(num_tests):
    fake_data = G(noise(num_test_samples)).detach()
    logger = Logger(model_name='loaded-GANART', data_name='Basics')
    logger.log_images(fake_data, num_test_samples, i, 0, 0);